In [1]:
import biolqm
import ginsim
import functools
import itertools
import logging
import operator
import random
from typing import Optional

import networkx

import pyboolnet.state_space
from pyboolnet import find_command
from pyboolnet import prime_implicants
from pyboolnet.digraphs import add_style_subgraphs, digraph2image
from pyboolnet.digraphs import find_ancestors, find_outdag
from pyboolnet.helpers import copy_json_data, save_json_data, open_json_data
from pyboolnet.helpers import dicts_are_consistent, merge_dicts
from pyboolnet.helpers import perc2str, divide_list_into_similar_length_lists
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.model_checking import model_checking
from pyboolnet.prime_implicants import copy_primes, remove_all_variables_except
from pyboolnet.prime_implicants import list_input_combinations, find_inputs
from pyboolnet.state_space import size_state_space, subspace2str
from pyboolnet.state_transition_graphs import UPDATE_STRATEGIES
from pyboolnet.temporal_logic import subspace2proposition
from pyboolnet.commitment_diagrams import _project_attractors
from pyboolnet.commitment_diagrams import _cartesian_product_of_diagrams
from pyboolnet.commitment_diagrams import _lift_attractors
import pyboolnet

import numpy as np
import pandas as pd

from pyboolnet import attractors
from pyboolnet import basins_of_attraction
from pyboolnet import commitment_diagrams
from pyboolnet import file_exchange
from pyboolnet import phenotypes
from pyboolnet import state_space
from pyboolnet import state_transition_graphs
from pyboolnet import network_generators
from pyboolnet import model_checking
from pyboolnet import temporal_logic
from pyboolnet import boolean_logic




from itertools import combinations
from colomoto import minibn
from colomoto_jupyter import tabulate
import os.path
import mpbn
import json
import copy

This notebook has been executed using the docker image `colomoto/colomoto-docker` built on `2023-12-30`

# MvReBnet: method for the identification of Multi-valued Refinements of Boolean network model

### Model loading
Loading of the BN model and any mutation of interest

In [3]:
model_name = input("Enter the name of the model: \n") # Name of the model

Enter the name of the model: 
 ML


In [4]:
from inputs import * # Python file with mutation, inits and names attrs

In [5]:
# BN model
primes = {}
primes["ASYN"] = file_exchange.bnet2primes("ASYN.bnet") # Asynchronous model
lqm = biolqm.load("ASYN.bnet")
bna = biolqm.to_minibn(lqm)
mbn = mpbn.MPBooleanNetwork(bna) # Most permissive model

In [6]:
# Mutation in the model
for g, v in mutations.items():
    prime_implicants.create_constants(primes["ASYN"], {g:v})
    mbn[g] = v

In [7]:
# Model visualization
bn = biolqm.to_ginsim(lqm)
ginsim.show(bn)

### Identification of the attractors

Determination of the attractors of the BN model under the asynchronous and most permissive scheme

In [8]:
# Separation of the stable states and cyclic attractors.
at_asyn = attractors.compute_attractors(primes["ASYN"], "asynchronous")
at_mp = list(mbn.attractors())

stable_states = {}
cyclic_attractors = {}

stable_states["ASYN"] = {i: attractor for i, attractor in enumerate(at_asyn["attractors"]) if not attractor["is_cyclic"]}
cyclic_attractors["ASYN"] = {i: attractor for i, attractor in enumerate(at_asyn["attractors"]) if attractor["is_cyclic"]}

stable_states["MP"] = {i: attractor for i, attractor in enumerate(at_mp) if "*" not in attractor.values()}
cyclic_attractors["MP"] = {i: attractor for i, attractor in enumerate(at_mp) if "*" in attractor.values()}

INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_mn9osnkg
INFO yes
INFO  working on minimal trapspace 1/5: 000000100010011
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=15, update=asynchronous, length=150, attempts=10
INFO trial 0
INFO start: 000000100010011
INFO end: 000000100010011
INFO created /tmp/pyboolnet_mgy02tbf
INFO is attractor state
INFO  working on minimal trapspace 2/5: 000100000000100
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=15, update=asynchronous, length=150, attempts=10
INFO trial 0
INFO start: 000100000000100
INFO end: 000100000000100
INFO created /tmp/pyboolnet_2xofm9tz
INFO is attractor state
INFO  working on minima

In [9]:
if not cyclic_attractors["ASYN"] and not cyclic_attractors["MP"]:
    print("There are no cyclic attractors")
elif cyclic_attractors["ASYN"] == cyclic_attractors["MP"]:
    print("The cyclic attractors are identical")
else:
    print("The cyclic attractors are different")

There are no cyclic attractors


## Propriety P: Asyncrhonous vs M.p.

In this part, we assess whether the most permissive dynamics can capture a specific reachability property, P, of interest. P may either be an increase in the sizes of the basins of attraction or a precise reachability observable only in the most permissive dynamics. In the case of precise reachability, a file dictating the desired reachability in the following format is necessary.


In [44]:
# Functions used in the notebook
def bool_to_jmp(state, m): # Translation of a Boolean state to a partial Jm.p. state    
    states = copy.deepcopy(state)
    if m not in ["MP", "ASYN"]:
        for k in m.split("_"):
            if k in states:
                v = states[k]
                for suffix in ['_a', '_b', '_c']:
                    states[k + suffix] = v
                del states[k]
    return states

def bool_to_mv(state, m): # Translation of a Boolean state to a partial Jm.p. state    
    states = copy.deepcopy(state)
    if m not in ["MP", "ASYN"]:
        for k in m.split("_"):
            if k in states:
                v = states[k]
                for suffix in ['_b1', '_b2']:
                    states[k + suffix] = v
                del states[k]
    return states

def sboa_df(ss_asyn, models): # Data frame of the sizes of the basins of attraction of fixed point
    rnames = [ss_asyn[i]["state"]['str'] for i in range(len(ss_asyn))]
    df = pd.DataFrame(columns=models, index=rnames)
    return df

def basin(attractors, primes, lm, model, sboa_df): # Calculates the basin of attraction
    init = {}
    for m in model[:-1]:
        if m == "ASYN":
            init[m] = "INIT TRUE"
        else:
            if lm == "mp":
                mn = [k[:-2] for k, _ in primes[m].items() if k.endswith("_c")]
                if mn:
                    clause = [f"({n}_a&{n}_b&{n}_c) | (!{n}_a&!{n}_b&!{n}_c)" for n in mn]
                    init[m] = f"INIT ({' & '.join(clause)})"
            else: 
                mn = [k[:-3] for k, _ in primes[m].items() if k.endswith("_b2")]
                if mn:
                    clause = [f"({n}_b1&{n}_b2) | (!{n}_b1&!{n}_b2)" for n in mn]
                    init[m] = f"INIT ({' & '.join(clause)})"

    for m in model[:-1]:
        if primes.get(m) is not None:
            size = len(primes["ASYN"])
            for i in range(len(attractors)):
                if lm == "mp":
                    attr = bool_to_jmp(attractors[i]["state"]["dict"], m)
                else: attr = bool_to_mv(attractors[i]["state"]["dict"], m)
                s = state_space.state2str(attr)
                specification = f"CTLSPEC EF({model_checking.subspace2proposition(primes[m], attr)})"
                _, accepting_states = model_checking.model_checking(primes[m],
                                                                    "asynchronous",
                                                                    init[m], 
                                                                    specification, 
                                                                    enable_accepting_states=True)
                sboa = accepting_states["INITACCEPTING_SIZE"]/2**size*100
                sboa_df.loc[s, m] = sboa

    for i in range(len(attractors)):
        states = list(mbn.reachable_from(attractors[i]["state"]["dict"], reversed= True))
        s = attractors[i]["state"]["str"]
        sboa_df.loc[s,"MP"] = len(states)/2**(size)*100          

    return sboa_df

def mc(primes, lm, model, mbn,inits, attrs): # Test the attractors reachability from the given initial states
    init = {}
    for m in model[:-1]:
        init[m] = {}
        if m == "ASYN":
            for n, i in inits.items():
                init[m][n] = f"INIT {subspace2proposition(primes[m], i)}"
        else:
            for n, i in inits.items():
                temp = copy.deepcopy(inits) 
                if lm == "mp":
                    init[m][n] = f"INIT {subspace2proposition(primes[m], bool_to_jmp(temp[n],m))}"
                else: init[m][n] = f"INIT {subspace2proposition(primes[m], bool_to_mv(temp[n],m))}"


    answer = {}
    for m in model[:-1]:
        answer[m] = {}
        for i in inits.keys():
            answer[m][i] = []
            for j in attrs.keys():
                if m == "ASYN":
                     attr = attrs[j]
                else:
                     if lm == "mp":
                         attr = bool_to_jmp(copy.deepcopy(attrs[j]),m)
                     else: attr = bool_to_mv(copy.deepcopy(attrs[j]),m)
                 
                specification = f"CTLSPEC EF({model_checking.subspace2proposition(primes[m],attr)})"
                res = model_checking.model_checking(primes[m],
                                                   "asynchronous",
                                                   init[m][i],
                                                   specification)
                n = j+" : "+str(res)
                answer[m][i].append(n)

    
    answer["MP"] = {}
    for n, i in inits.items():
        answer["MP"][n] = []
        for j in attrs.keys():
            res = j+" : "+str(mbn.reachability(i, attrs[j]))
            answer["MP"][n].append(res)
            
    return answer

In [45]:
# Checking if the basins of attraction are different
df = sboa_df(stable_states["ASYN"], ["ASYN","MP"])
basin_asyn_vs_mp = basin(stable_states["ASYN"], primes, "mp", ["ASYN","MP"], df)
basin_asyn_vs_mp.to_csv(model_name+"_MvReBnet_basin_asyn_vs_mp_basin_sizes.csv")

INFO created /tmp/pyboolnet_yhywlc0i
INFO created /tmp/pyboolnet_kumfv0z4
INFO created /tmp/pyboolnet_dje4af9h
INFO created /tmp/pyboolnet_603rjwf0
INFO created /tmp/pyboolnet_aok_wid8


In [12]:
if not (basin_asyn_vs_mp["ASYN"].all == basin_asyn_vs_mp["MP"].all):
    print("The most permissive dynamics shows an increase in the sizes of the basin of attraction")
else:
    print("No difference in the sizes of basin of attraction")

The most permissive dynamics shows an increase in the sizes of the basin of attraction


In [46]:
basin_asyn_vs_mp

,ASYN,MP
000000000000000,61.71875,81.982422
000000011000100,16.40625,63.28125
000000100010011,69.604492,86.083984
000001100000011,97.65625,98.4375
000100000000100,70.483398,78.271484


In [47]:
# Checking if a reahcability of interest is recovered by the most permissive scheme
mc_asyn_vs_mp = mc(primes, "mp", ["ASYN","MP"], mbn, inits, attrs)

INFO created /tmp/pyboolnet_w4f00nep
INFO created /tmp/pyboolnet_9zmloe_k
INFO created /tmp/pyboolnet_ku8trgi9
INFO created /tmp/pyboolnet_snj1hovf
INFO created /tmp/pyboolnet_wgc55c07
INFO created /tmp/pyboolnet_nt0zimfs
INFO created /tmp/pyboolnet_lx3o8fo7
INFO created /tmp/pyboolnet__l4f8m8n
INFO created /tmp/pyboolnet_kunfkpzi
INFO created /tmp/pyboolnet_o3vpgyzg
INFO created /tmp/pyboolnet_f7ewnp65
INFO created /tmp/pyboolnet_ziqbshep
INFO created /tmp/pyboolnet_okda18ec
INFO created /tmp/pyboolnet_myz7_xp8
INFO created /tmp/pyboolnet_56k6hdrv
INFO created /tmp/pyboolnet_i0n0uq9v
INFO created /tmp/pyboolnet_hldv622x
INFO created /tmp/pyboolnet_307hkzcc
INFO created /tmp/pyboolnet_s3zp76td
INFO created /tmp/pyboolnet_sacu5alq
The most permissive dynamics captures the reachability of interest, not captured by the asynchronous dynamics


In [48]:
if mc_asyn_vs_mp["ASYN"] != mc_asyn_vs_mp["MP"]:
    print("The most permissive dynamics captures the reachability of interest, not captured by the asynchronous dynamics")
else: print("No difference for the reachability of interest")

The most permissive dynamics captures the reachability of interest, not captured by the asynchronous dynamics


### Propriety P: M.p vs Partial Jm.p.

In this part, we assess which partial Jm.p. dynamics can capture the reachability property P.


In [15]:
# List of admissible np nodes: no output or auto-inhibited nodes
# Extract nodes with self-inhibition from 'primes["ASYN"]'
self_in = [i for i in primes["ASYN"] if f"!{i}" in str(bna[i])]
# Find outputs in 'primes["ASYN"]'
outputs = prime_implicants.find_outputs(primes['ASYN'])
# Filter out nodes that are neither outputs nor have self-inhibition
mp_nodes = [item for item in primes['ASYN'] if item not in outputs and item not in self_in]

In [49]:
nb_mp_nodes = input("Enter the number of mp node(s) you want to test: \n")

Enter the number of mp node(s) you want to test: 
 4


In [37]:
# List of sets J depending on the size defined
jsets = list(combinations(mp_nodes, int(nb_mp_nodes))) # Combinations of mp nodes
models = ["_".join(map(str, i)) for i in jsets]
models.insert(0, "ASYN")
models.append("MP")

Use the list "jsets" as input to generate the partial mp dynamics in Biolqm. The resulting models should be in the same directory.

In [38]:
# jm.p. model loading
for m in models[1:-1]:
    path = m + "_mp.bnet"
    if os.path.isfile(path) == True:
        primes[m] = file_exchange.bnet2primes(path)
        for g, v in mutations.items():
            if {g}.issubset(m.split("_")) == True:
                prime_implicants.create_constants(primes[m], {g+"_a":v, g+"_b":v, g+"_c":v})
            else:
                prime_implicants.create_constants(primes[m], {g:v})

In [19]:
# Sizes of the basin of attraction under the partial most permissive scheme
df = sboa_df(stable_states["ASYN"], models[1:-1])
basin_sizes = basin(stable_states["ASYN"], primes, "mp", models[1:-1], df)
basin_sizes.to_csv(model_name+"_MvReBnet_"+nb_mp_nodes+"mp_basin_sizes.csv")

INFO created /tmp/pyboolnet_9pkpop1v
INFO created /tmp/pyboolnet_ajdvqxus
INFO created /tmp/pyboolnet_e907jduu
INFO created /tmp/pyboolnet_43av6u5i
INFO created /tmp/pyboolnet_ugif2yie
INFO created /tmp/pyboolnet_36gm2872
INFO created /tmp/pyboolnet_zzsi93xp
INFO created /tmp/pyboolnet_hh6pg60y
INFO created /tmp/pyboolnet_1dzq2nai
INFO created /tmp/pyboolnet_hork4mcu
INFO created /tmp/pyboolnet_be_fx6a2
INFO created /tmp/pyboolnet_hzbb_laf
INFO created /tmp/pyboolnet_uu1rgde7
INFO created /tmp/pyboolnet_twlz7h8w
INFO created /tmp/pyboolnet_nfq3tn7t
INFO created /tmp/pyboolnet_brfq73xx
INFO created /tmp/pyboolnet_0i3tvk3t
INFO created /tmp/pyboolnet_jouclqdq
INFO created /tmp/pyboolnet_rgxp3x_u
INFO created /tmp/pyboolnet_pnp84rdu
INFO created /tmp/pyboolnet_ap4intgh
INFO created /tmp/pyboolnet_ob1st72w
INFO created /tmp/pyboolnet_d7o0rmfz
INFO created /tmp/pyboolnet_731sz0lw
INFO created /tmp/pyboolnet_ibo_nhz3
INFO created /tmp/pyboolnet_n79f6cgp
INFO created /tmp/pyboolnet_i_8n0gao
I

In [39]:
# Checking if a reachability of interest is recovered under the partial most permissive scheme
answer = mc(primes, "mp", models, mbn, inits, attrs)
with open(model_name+"_MvReBnet_"+nb_mp_nodes+'model_checking.txt', 'w') as f:
    print(answer, file=f)

INFO created /tmp/pyboolnet_obw5scgp
INFO created /tmp/pyboolnet_4ijzz3gg
INFO created /tmp/pyboolnet_kwyuh5q6
INFO created /tmp/pyboolnet_il9tkmtv
INFO created /tmp/pyboolnet_hupesnnn
INFO created /tmp/pyboolnet_okyzklc1
INFO created /tmp/pyboolnet_nl9pxv3m
INFO created /tmp/pyboolnet_syuyf57f
INFO created /tmp/pyboolnet_a9aqu4ba
INFO created /tmp/pyboolnet_a8ifkf4d
INFO created /tmp/pyboolnet_ohyq36ut
INFO created /tmp/pyboolnet_hadc64pe
INFO created /tmp/pyboolnet_9r5iw0lp
INFO created /tmp/pyboolnet_f1vqsy1z
INFO created /tmp/pyboolnet_ff0c4ctz
INFO created /tmp/pyboolnet_6wxgyy0f
INFO created /tmp/pyboolnet_cu8bj09y
INFO created /tmp/pyboolnet_yf2ipi4o
INFO created /tmp/pyboolnet_kujhev74
INFO created /tmp/pyboolnet__azikit5
INFO created /tmp/pyboolnet_dvhc5e2x
INFO created /tmp/pyboolnet_11vuh67v
INFO created /tmp/pyboolnet_t7ix0x0l
INFO created /tmp/pyboolnet_5rktpqsf
INFO created /tmp/pyboolnet_yugs3ihs
INFO created /tmp/pyboolnet_prmbtmgr
INFO created /tmp/pyboolnet_s2hh_6gi
I

In [40]:
# Identifying models with different attractors reachability than the asynchronous dynamics
asyn_diff = [m for m in models[1:] if answer["ASYN"] != answer[m]]
asyn_diff

['Bclaf1_Cebpa_Egr1_Fli1',
 'Bclaf1_Cebpa_Egr1_Gata2',
 'Bclaf1_Cebpa_Egr1_Spi1',
 'Bclaf1_Cebpa_Fli1_Gata1',
 'Bclaf1_Cebpa_Fli1_Gata2',
 'Bclaf1_Cebpa_Fli1_Ikzf1',
 'Bclaf1_Cebpa_Fli1_Junb',
 'Bclaf1_Cebpa_Fli1_Klf1',
 'Bclaf1_Cebpa_Fli1_Myc',
 'Bclaf1_Cebpa_Fli1_Spi1',
 'Bclaf1_Cebpa_Fli1_Zfpm1',
 'Bclaf1_Cebpa_Gata1_Gata2',
 'Bclaf1_Cebpa_Gata1_Spi1',
 'Bclaf1_Cebpa_Gata2_Ikzf1',
 'Bclaf1_Cebpa_Gata2_Junb',
 'Bclaf1_Cebpa_Gata2_Klf1',
 'Bclaf1_Cebpa_Gata2_Myc',
 'Bclaf1_Cebpa_Gata2_Spi1',
 'Bclaf1_Cebpa_Gata2_Zfpm1',
 'Bclaf1_Cebpa_Ikzf1_Spi1',
 'Bclaf1_Cebpa_Junb_Spi1',
 'Bclaf1_Cebpa_Klf1_Spi1',
 'Bclaf1_Cebpa_Myc_Spi1',
 'Bclaf1_Cebpa_Spi1_Zfpm1',
 'Bclaf1_Egr1_Fli1_Gata1',
 'Bclaf1_Egr1_Fli1_Gata2',
 'Bclaf1_Egr1_Fli1_Ikzf1',
 'Bclaf1_Egr1_Fli1_Junb',
 'Bclaf1_Egr1_Fli1_Klf1',
 'Bclaf1_Egr1_Fli1_Myc',
 'Bclaf1_Egr1_Fli1_Spi1',
 'Bclaf1_Egr1_Fli1_Zfpm1',
 'Bclaf1_Egr1_Gata1_Gata2',
 'Bclaf1_Egr1_Gata1_Spi1',
 'Bclaf1_Egr1_Gata2_Ikzf1',
 'Bclaf1_Egr1_Gata2_Junb',
 'Bclaf1_Egr1_Ga

In [41]:
# Identifying models from 'diff' with identical attractors reachability as the most permissive dynamics
mp_same = [d for d in asyn_diff if answer["MP"] == answer[d]]
mp_same

['Egr1_Fli1_Gata1_Gata2', 'MP']